# Filtros FIR por janelamento

Nesse caderno é mostrado os principais tipos de filtros FIR por janelamento.

In [1]:
# Importa bibliotecas
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import IPython.display as ipd
from scipy.io import wavfile
from scipy import signal
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True) #initialize on notebook
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import plotly.tools as tls #matplotlib to plotly

## Filtro passa-baixas ideal
A resposta ao impulso de um filtro ideal é dada por:

In [2]:
# Filtro ideal no domínio da frequência e no domínio do tempo
wc=[]
def filtroIdealFIR (wc0=1.5):
    global wc
    wc = wc0
    #wc = 1.5 #frequência de corte (rad)
    N = 501 #num amostras

     #filtro passa-baixas ideal: domínio da frequência
    omega = np.linspace(-np.pi, np.pi, N)
    x_fft = np.empty(N)
    x_fft[abs(omega)<wc] = 1 #banda de passagem
    x_fft[abs(omega)>wc] = 0 #banda de corte

    plt.figure(figsize=(12,1.5))
    plt.plot(omega, x_fft, c='orange', lw=5)
    plt.xlabel(r'$\omega$')
    plt.title('Domínio da Frequência')
    plt.xlim((-np.pi, np.pi))

    #filtro passa-baixas ideal: domínio do tempo
    n = np.linspace(-10, 10, 101)
    x = np.empty_like(n)
    x[n == 0] = wc/np.pi
    x[n != 0] = (1/np.pi/n[n != 0])*np.sin(wc*n[n != 0])

    plt.figure(figsize=(12,3))
    plt.plot(n, x, c='grey', lw=.5)
    plt.stem(n, x, 'k', markerfmt='ko', use_line_collection=True)
    plt.xlabel(r'$t$')
    plt.title('Domínio do Tempo (discreto)')
    plt.xlim((-10, 10))

    plt.show()

# Slider
interact(filtroIdealFIR, wc0=widgets.FloatSlider(min=0.0,max=np.pi,step=.1,value=1.5));

interactive(children=(FloatSlider(value=1.5, description='wc0', max=3.141592653589793), Output()), _dom_classe…

## Janelas
Abaixo estão colocadas algumas das principais janelas usadas em filtro FIR.

In [3]:
# Funções de janelamento para filtros FIR
def janelamentosFIR(windowType, numTaps=55):
    N = 101
    n = np.linspace(-10, 10, N)
    
    x = np.empty_like(n)
    x[n == 0] = wc/np.pi
    x[n != 0] = (1/np.pi/n[n != 0])*np.sin(wc*n[n != 0])
    
    wn = signal.get_window(window=windowType, Nx=N)
    # boxcar, triang, blackman, hamming, hann, bartlett, flattop,
    # parzen, bohman, blackmanharris, nuttall, barthann, kaiser (needs beta),
    # gaussian (needs standard deviation), general_gaussian (needs power, width),
    # slepian (needs width), dpss (needs normalized half-bandwidth),
    # chebwin (needs attenuation), exponential (needs decay scale), tukey (needs taper fraction)
    
    #FIR filter
    num = signal.firwin(numtaps=numTaps, cutoff = .3, window = windowType)
    omega, Hz = signal.freqz(num)
    
    #faixa de transição entre -20dB e -3dB (para preenchimento no gráfico)
    omg_tr = omega[(abs(Hz)<10**(-3/20)) & (abs(Hz)>10**(-20/20))]
    
    plt.figure(figsize=(10,2))
    plt.subplot(121)
    plt.title('Função Janela')
    plt.plot(n, wn, lw=2)
    plt.xlim((-10, 10))
    plt.ylim((-.1, 1.1))
    plt.xticks([])
    plt.xlabel(r'$n$')
    # plt.stem(n, wn, 'k', markerfmt='ko', lw=5)
    
#     plt.figure(figsize=(10,2))
    plt.subplot(122)
    plt.title('Resposta Filtro Ideal')
    plt.plot(n, x, c='orange', lw=2)
    plt.xlim((-10, 10))
    plt.xticks([])
    plt.xlabel(r'$n$')
    
    plt.figure(figsize=(10,2))
    plt.title(r'Resposta ao Impulso Filtro $h(n)$')
    plt.ylabel(r'$h(n)$')
    plt.plot(n, wn*x, c='red', lw=2)
    plt.xlim((-10, 10))
    plt.xlabel(r'$n$')
    
    plt.figure(figsize=(10,2))
    plt.title('$Módulo |H(e^{j\omega})|$')
    plt.ylabel('$|H(e^{j\omega})|$')
    plt.plot(omega, abs(Hz), c='green', lw=3)
    plt.fill([omg_tr[-1],omg_tr[0],omg_tr[0],omg_tr[-1]], [0, 0, 1, 1], "0.7")
#     plt.plot(omega[abs(Hz)<10**(-3/20)], abs(Hz)[abs(Hz)<10**(-3/20)], c='red', lw=3)
#     plt.plot(omega, 20*np.log10(abs(Hz)), c='green', lw=3)
#     plt.ylabel(r'dB')
    plt.xlabel(r'$\omega$')
    plt.xlim((0, np.pi))
    
    plt.figure(figsize=(10,2))
    plt.title('Fase $H(e^{j\omega})$')
    plt.ylabel(r'$\measuredangle H(e^{j\omega})$')
    plt.plot(omega, np.unwrap(np.angle(Hz)), c='orange', lw=3)
    plt.xlabel(r'$\omega$')
    plt.xlim((0, np.pi))
    
    plt.show()

# Mecanismo de interação
interact(janelamentosFIR,
         windowType=['boxcar', 'triang', 'blackman', 'hamming', 'hann', 'bartlett',
         'flattop', 'parzen', 'bohman', 'blackmanharris', 'nuttall', 'barthann'],
          numTaps=widgets.IntSlider(min=5,max=1000,step=5,value=20));
# boxcar, triang, blackman, hamming, hann, bartlett, flattop,
# parzen, bohman, blackmanharris, nuttall, barthann, kaiser (needs beta),
# gaussian (needs standard deviation), general_gaussian (needs power, width),
# slepian (needs width), dpss (needs normalized half-bandwidth),
# chebwin (needs attenuation), exponential (needs decay scale), tukey (needs taper fraction)

interactive(children=(Dropdown(description='windowType', options=('boxcar', 'triang', 'blackman', 'hamming', '…

## Projeto de Filtros FIR: janela de Kaiser
Aqui são mencionadas técnicas de projeto usando as janelas de Kaiser e Dolph-Chebyshev.

In [4]:
# Funções de janelamento para filtros FIR
def janelamentosFIR(ripple_db=1, width=50, fs=2*314):
    N = 101
    n = np.linspace(-10, 10, N)

#     x = np.empty_like(n)
#     x[n == 0] = wc/np.pi
#     x[n != 0] = (1/np.pi/n[n != 0])*np.sin(wc*n[n != 0])
    
#     wn = signal.get_window(window=windowType, Nx=N)
    # boxcar, triang, blackman, hamming, hann, bartlett, flattop,
    # parzen, bohman, blackmanharris, nuttall, barthann, kaiser (needs beta),
    # gaussian (needs standard deviation), general_gaussian (needs power, width),
    # slepian (needs width), dpss (needs normalized half-bandwidth),
    # chebwin (needs attenuation), exponential (needs decay scale), tukey (needs taper fraction)
    
    #FIR filter
    numTaps, beta = signal.kaiserord(ripple=ripple_db, width=width/(0.5*fs), )
    num = signal.firwin(numtaps=numTaps, cutoff = 100, window = ('kaiser', beta), fs=fs)
    omega, Hz = signal.freqz(num)
    
    plt.figure(figsize=(10,2))
    plt.ylabel('$|H(e^{j\omega})|$')
    plt.plot(omega, abs(Hz), c='green', lw=3)
#     plt.plot(omega, 20*np.log10(abs(Hz)), c='green', lw=3)
#     plt.ylabel(r'dB')
    plt.ylim((-.1,1.3))
    plt.xlabel(r'$\omega$')
    
    plt.figure(figsize=(10,2))
    plt.ylabel('$\measuredangle H(e^{j\omega})$')
    plt.plot(omega, np.unwrap(np.angle(Hz)), c='orange', lw=3)
    plt.xlabel(r'$\omega$')
    
    plt.show()
    print('numTaps = %i\nbeta = %.2f'%(numTaps, beta))

# Mecanismo de interação
interact(janelamentosFIR,
         ripple_db=widgets.FloatSlider(min=2,max=20,step=.1,value=10),
         width=widgets.FloatSlider(min=0.01,max=10,step=.005,value=2));
# kaiser (needs beta),
# gaussian (needs standard deviation), general_gaussian (needs power, width),
# slepian (needs width), dpss (needs normalized half-bandwidth),
# chebwin (needs attenuation), exponential (needs decay scale), tukey (needs taper fraction)

interactive(children=(FloatSlider(value=10.0, description='ripple_db', max=20.0, min=2.0), FloatSlider(value=2…